Rerun the g band exposures.

Remove detections with estimated variance of greater than 1000 mas^2 and less than 100 mas^2. These detections seem to have mis-estimated error according to gBandTest.ipynb

In [1]:
import sys
import os
import shutil
import glob
import traceback
import warnings
warnings.filterwarnings("ignore")

import GPRutils
import vK2KGPR
import plotGPR
import vonkarmanFT as vk

import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from astropy.modeling import models, fitting

from importlib import reload

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
gBand = sorted(np.array([
    474262,
    361580,
    474265,
    696547,
    484483,
    367484,
    369801,
    576863,
    791229,
    791593,
    791640,
    579816
]))
gBand

[361580,
 367484,
 369801,
 474262,
 474265,
 484483,
 576863,
 579816,
 696547,
 791229,
 791593,
 791640]

In [ ]:
for expNum in gBand[:3]:
    
    expFile = os.path.join("/home/fortino/gBand_useRMS", str(expNum))
    try:
        os.mkdir(expFile)
    except FileExistsError:
        shutil.rmtree(expFile)
        os.mkdir(expFile)

    sys.stderr = open(os.path.join(expFile, "err.err"), "a+")
    sys.stdout = open(os.path.join(expFile, "out.out"), "a+")
    
    try:
        dataC = GPRutils.dataContainer()

        dataC.load(expNum, downselect=0.5, useRMS=True)

        GP = vK2KGPR.vonKarman2KernelGPR(dataC, printing=True, outDir=expFile)
        GP.fitCorr()
        GP.fit(dataC.fitCorrParams)
        dataC.postFitCorr_sigmaClip(GP)
        GP.optimize(v0=dataC.fitCorrParams)
        dataC.JackKnife(GP)
        dataC.saveFITS(expFile)
    except:
        traceback.print_exc()

In [ ]:
# for expNum in gBand[:3]:
    
#     expFile = os.path.join("/home/fortino/gBand_useCov", str(expNum))
#     try:
#         os.mkdir(expFile)
#     except FileExistsError:
#         shutil.rmtree(expFile)
#         os.mkdir(expFile)

#     sys.stderr = open(os.path.join(expFile, "err.err"), "a+")
#     sys.stdout = open(os.path.join(expFile, "out.out"), "a+")
    
#     try:
#         dataC = GPRutils.dataContainer()

#         dataC.load(expNum, downselect=0.5, useRMS=True, useCov=True)

#         GP = vK2KGPR.vonKarman2KernelGPR(dataC, printing=True, outDir=expFile)
#         GP.fitCorr()
#         GP.fit(dataC.fitCorrParams)
#         dataC.postFitCorr_sigmaClip(GP)
#         GP.optimize(v0=dataC.fitCorrParams)
#         dataC.JackKnife(GP)
#         dataC.saveFITS(expFile)
#     except:
#         traceback.print_exc()

In [ ]:
for expNum in gBand[:3]:
    
    expFile = os.path.join("/home/fortino/gBand_minmaxErr", str(expNum))
    try:
        os.mkdir(expFile)
    except FileExistsError:
        shutil.rmtree(expFile)
        os.mkdir(expFile)

    sys.stderr = open(os.path.join(expFile, "err.err"), "a+")
    sys.stdout = open(os.path.join(expFile, "out.out"), "a+")
    
    try:
        dataC = GPRutils.dataContainer()

        dataC.load(expNum, downselect=0.5, minDESErr=1*u.mas**2, maxDESErr=1000*u.mas**2)

        GP = vK2KGPR.vonKarman2KernelGPR(dataC, printing=True, outDir=expFile)
        GP.fitCorr()
        GP.fit(dataC.fitCorrParams)
        dataC.postFitCorr_sigmaClip(GP)
        GP.optimize(v0=dataC.fitCorrParams)
        dataC.JackKnife(GP)
        dataC.saveFITS(expFile)
    except:
        traceback.print_exc()